In [ ]:
#Importar librerias necesarias#
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import csv
import json
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler


from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

#TOKENS DE ACCESO#
CONSUMER_TOKEN = "gzNjoVdsjT3EZPet3cbhRnA2Q"
CONSUMER_SECRET = "LLZmHMfiK4hmzUjlXaAw4QseEGw4YfMOvz2jVRpM6DAIqofc2V"
ACCESS_TOKEN = "389120518-j1VFKmSuI92XqYIHAkNc5wKuM85j4OTvfvTg7mY4"
ACCESS_TOKEN_SECRET = "nS1Qo3nH6VPUKBMHn8SbcOV8HPWDJdgHCvrlLaQREHhOz"

#AUTENTIFICACION#
auth = tweepy.OAuthHandler(CONSUMER_TOKEN, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

#Analisis de sentimientos inicial#
def porcentage (part,whole):
    return 100 * float(part)/float(whole)

keyword = input ("Introduce la palabra a buscar: ")
noOfTweet = int(input("Introduce el nº de tweets a analizar: "))

tweetsP =tweepy.Cursor (api.search, q=keyword ,lang='en',until='2021-04-19').items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
date_list = []
user_list = []
location_list = []
posdate_list = []
posuser_list = []
poslocation_list = []
neudate_list = []
neuuser_list = []
neulocation_list = []
negdate_list = []
neguser_list = []
neglocation_list = []
for tweet in tweetsP:
    date_list.append(tweet.created_at)
    tweet_list.append(tweet.text)
    user_list.append(tweet.user.screen_name)
    location_list.append(tweet.user.location)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer() .polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negdate_list.append(tweet.created_at)
        negative_list.append(tweet.text)
        neguser_list.append(tweet.user.screen_name)
        neglocation_list.append(tweet.user.location)
        negative += 1
    elif pos > neg:
        posdate_list.append(tweet.created_at)
        positive_list.append(tweet.text)
        posuser_list.append(tweet.user.screen_name)
        poslocation_list.append(tweet.user.location)
        positive += 1
    elif pos == neg:
        neudate_list.append(tweet.created_at)
        neutral_list.append(tweet.text)
        neuuser_list.append(tweet.user.screen_name)
        neulocation_list.append(tweet.user.location)
        neutral += 1
positive = porcentage(positive, noOfTweet)
negative = porcentage(negative, noOfTweet)
neutral = porcentage(neutral, noOfTweet)
polarity = porcentage(polarity, noOfTweet)
positive = format(positive,'.1f')
negative = format(negative,'.1f')
neutral = format(neutral,'.1f')

In [ ]:
#Nº de Tweets recolectados (Total, Positivo, Negativo, Neutro)#
df = pd.DataFrame ()

In [ ]:
#Construcción del DataFrame#
df['TweetAt'] = date_list

In [ ]:
df['ScreenName'] = user_list

In [ ]:
df['Location'] = location_list

In [ ]:
df['OriginalTweet'] = tweet_list

In [ ]:
dfpos = pd.DataFrame ()

In [ ]:
dfpos['TweetAT'] = posdate_list

In [ ]:
dfpos['ScreenName'] = posuser_list

In [ ]:
dfpos['Location'] = poslocation_list

In [ ]:
dfpos['OriginalTweet'] = positive_list

In [ ]:
dfneu = pd.DataFrame ()

In [ ]:
dfneu['TweetAT'] = neudate_list

In [ ]:
dfneu['ScreenName'] = neuuser_list

In [ ]:
dfneu['Location'] = neulocation_list

In [ ]:
dfneu['OriginalTweet'] = neutral_list

In [ ]:
dfneg = pd.DataFrame ()

In [ ]:
dfneg['TweetAT'] = negdate_list

In [ ]:
dfneg['ScreenName'] = neguser_list

In [ ]:
dfneg['Location'] = neglocation_list

In [ ]:
dfneg['OriginalTweet'] = negative_list

In [ ]:
#Agregar a nuestra tabla principal el sentimiento de cada tweet#
for index, row in df['OriginalTweet'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        df.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        df.loc[index, 'sentiment'] = "positive"
    else:
        df.loc[index, 'sentiment'] = "neutral"
    df.loc[index, 'neg'] = neg
    df.loc[index, 'neu'] = neu
    df.loc[index, 'pos'] = pos
    df.loc[index, 'compound'] = comp
df.head(10)

In [ ]:
#Preprocesamiento de los datos#
#Eliminacion de usuarios (@)#

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

# create new column with removed @user
df['Tweet'] = np.vectorize(remove_pattern)(df['OriginalTweet'], '@[\w]*')


In [ ]:
#Eliminacion de enlaces#

import re
df['Tweet'] = df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df.head(3)

In [ ]:
#Eliminacion de caracteres especiales, signos de puntuación  y nº#

# remove special characters, numbers, punctuations
df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')
df.head(5)

In [ ]:
Tokenizacion#
df['tokenizado'] = df['Tweet'].apply(lambda x: x.split())
df.columns

In [ ]:
#Guardar el DF para trabajar posteriormente con el#
df.to_csv("C:\Tweets\Johnson1904puro.csv", header=True, sep='\t')

In [ ]:
#Importar librerias necesarias#
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import csv
import json
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import glob
import seaborn as sns
import re

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
#Importación de los distintos archivos de recolección en uno solo#
path = r'C:\AllTweets' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep='\t' )
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
#Información sobre nuestro dataframe#
frame.info()

In [ ]:
#Dividir la columna "TweetAt" para separarlo en fecha y hora#
new = frame["TweetAt"].str.split(" ", n = 1, expand = True)
frame["Date"]= new[0]
frame["Hour"]= new[1]
frame.drop(columns =["TweetAt"], inplace = True)
frame.info()

In [ ]:
#Gráfico fechas#
fechas = frame['Date'].value_counts()
graf = sns.catplot("Date", data=frame, kind="count", height=15)
graf.fig.suptitle('Frecuencia de fechas')
graf.savefig('C:\AllTweets\Todasfechas.png')

In [ ]:
#Gráfico sentimientos#
sentimiento = frame['sentiment'].value_counts()
graf = sns.catplot("sentiment", data=frame, kind="count", height=8)
graf.fig.suptitle('Frecuencia de sentimientos')
graf.savefig('C:\AllTweets\Todassentimiento.png')

In [ ]:
#Eliminación de columnas innecesarias#
frame = frame.drop(columns=['Unnamed: 0', 'neg', 'neu', 'pos', 'compound', 'Tweet', 'tokenizado','Hour'])
frame.head(10)

In [ ]:
#PREPROCESAMIENTO DE LOS DATOS#
#Eliminación de los usuarios mencionados en los tweet#
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

frame['Tweet'] = np.vectorize(remove_pattern)(frame['OriginalTweet'], '@[\w]*')
frame.head(2).to_excel('C:\AllTweets\PruebaP.xlsx')

In [ ]:
#Eliminación de las URL#
import re
frame['Tweet'] = frame['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
frame.head(2).to_excel('C:\AllTweets\EliminaciónURL.xlsx')

In [ ]:
#Eliminacion de signos de puntuación, nº y caracteres especiales#
frame['Tweet'] = frame['Tweet'].str.replace('[^a-zA-Z#]+',' ')
frame.head(2).to_excel('C:\AllTweets\Eliminaciónpuntuacion.xlsx')

In [ ]:
#Eliminación de "Short Words"#
frame['Tweet'] = frame['Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
frame.head(10).to_excel('C:\AllTweets\Eliminaciónshortwords.xlsx')

In [ ]:
#Tokenizacion#
token_tweet = frame['Tweet'].apply(lambda x: x.split())

In [ ]:
#Derivacion#
from nltk.stem.porter import *
stemmer = PorterStemmer()

token_tweet = token_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
#Sustituir la antiguo columna de Tweet por una nueva tokenizada y derivada#
frame['Tweet'] = token_tweet
frame.head(10).to_excel('C:\AllTweets\Stemming.xlsx')

In [ ]:
#Eliminación de las lineas que contegan el mismo tweet#
frame.drop_duplicates(subset=['Tweet'],inplace=True)
frame.info()

In [ ]:
#Wordcloud para todos los tweets#
all_words = ' '.join([text for text in frame['Tweet']])

from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.savefig('C:\AllTweets\Cloudall.png')

In [ ]:
#WordCloud para tweets positivos#
positive_words = ' '.join([text for text in frame['Tweet'][frame['sentiment'] == 'positive']])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(positive_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.savefig('C:\AllTweets\Cloudpositive.png')

In [ ]:
#WordCloud para tweets negativos#
negative_words = ' '.join([text for text in frame['Tweet'][frame['sentiment'] == 'negative']])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(negative_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.savefig('C:\AllTweets\Cloudnegative.png')

In [ ]:
#WordCloud para tweets neutrales#
neutral_words = ' '.join([text for text in frame['Tweet'][frame['sentiment'] == 'neutral']])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(neutral_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.savefig('C:\AllTweets\Cloudneutral.png')

In [ ]:
#Recolección de hastags#
def hashtag_extract(x):
    hashtags = []
    for i in x:
        ht = re.findall(r'#(\w+)', i)
        hashtags.append(ht)
    return hashtags

In [ ]:
HT_Positive = hashtag_extract(frame['OriginalTweet'][frame['sentiment'] == 'positive'])
HT_Neutral = hashtag_extract(frame['OriginalTweet'][frame['sentiment'] == 'neutral'])
HT_Negative = hashtag_extract(frame['OriginalTweet'][frame['sentiment'] == 'negative'])

In [ ]:
HT_Positive = sum(HT_Positive, [])
HT_Neutral = sum(HT_Neutral, [])
HT_Negative = sum(HT_Negative,[])

In [ ]:
#Gráfico para hastafs de tweets positivos#
a = nltk.FreqDist(HT_Positive)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count' : list(a.values())})

d = d.nlargest(columns = 'Count', n = 10)

plt.figure(figsize = (16,5))
ax = sns.barplot(data =d, x = 'Hashtag', y = 'Count')
plt.savefig('C:\AllTweets\Hastagpositivo.png')

In [ ]:
#Graáfico para hashtags de tweets neutrales#
a = nltk.FreqDist(HT_Neutral)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count' : list(a.values())})

d = d.nlargest(columns = 'Count', n = 10)

plt.figure(figsize = (16,5))
ax = sns.barplot(data =d, x = 'Hashtag', y = 'Count')
plt.savefig('C:\AllTweets\Hastagneutral.png')

In [ ]:
#Graáfico para hashtags de tweets neutrales#
a = nltk.FreqDist(HT_Negative)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count' : list(a.values())})

d = d.nlargest(columns = 'Count', n = 10)

plt.figure(figsize = (16,5))
ax = sns.barplot(data =d, x = 'Hashtag', y = 'Count')
plt.savefig('C:\AllTweets\Hastagnegaivo.png')

In [ ]:
#Dejar solo las columnas con las que vamos a trabjar a partir de ahora (Tweet depurado y sentimiento)#
new_frame = frame[['Tweet', 'sentiment']]

In [ ]:
#Eliminar mayusculas#

new_frame["Tweet"] = new_frame["Tweet"].str.lower().str.split()

In [ ]:
#Eliminacion de StopWords#

from nltk.corpus import stopwords
stop = stopwords.words('english')
new_frame['Tweet'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
#Unir los tokens en una frase#
new_frame['Tweet'] = new_frame['Tweet'].apply(lambda x: ' '.join(x))

In [ ]:
#Comprobar ele tamaño de nuestro nuevo df#
new_frame.shape

In [ ]:
#Detectar si hay algun valor vacio#

new_frame.isnull().sum()

In [ ]:
#Proceso de modelizacion#
#Diviendo nuestro dataset en train y test#
from sklearn.model_selection import train_test_split

train,valid = train_test_split(new_frame,test_size = 0.2,random_state=42, stratify = new_frame.sentiment.values) #stratification means that the train_test_split method returns training and test subsets that have the same proportions of class labels as the input dataset.
print("train shape : ", train.shape)
print("valid shape : ", valid.shape)

In [ ]:
#Aplicando COUNTVECTORIZER#
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

X_train = vectorizer.fit_transform(train.Tweet.values)
X_valid = vectorizer.transform(valid.Tweet.values)

y_train = train.sentiment.values
y_valid = valid.sentiment.values

print("X_train.shape : ", X_train.shape)
print("X_train.shape : ", X_valid.shape)
print("y_train.shape : ", y_train.shape)
print("y_valid.shape : ", y_valid.shape)

In [ ]:
#TEST TUNEADO DE PARAMETROS RF#
# Ver que parametros podemos modificar de nuestro algoritmo#
from pprint import pprint

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
print('Parametros en uso')
pprint(rf.get_params())

In [ ]:
#Crear parrilla de hyperparametros Random#
#Nº de arboles en el RF#
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#Nº nº de caracteristicas a tener en cuenta para cada division#
max_features = ['sqrt']
#Numero maximo de niveles en el arbol#
max_depth = [int(x) for x in np.linspace(10,110, num = 11)]
#Tamaño minimo de muestras para dividir un nodo#
min_samples_split = [2, 5, 7]
#Tamaño minimo de hoja#
min_samples_leaf = [ 2, 4]
#Metodo de seleccion de muestra para entrenar cada arbol#
bootstrap = [True, False]
#Criterio#
criterion = ['gini', 'entropy']

#Creando la parrilla una vez indicadas los diferentes tuneados#
random_grid = {'n_estimators': n_estimators,'max_features': max_features, 'max_depth': max_depth, 'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap, 'criterion': criterion}

pprint(random_grid)

In [ ]:
#Random Search Trainning#
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf_clf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = random_grid, n_iter = 100, cv = 5,verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.cv_results_

In [ ]:
#GRID SEARCH#
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV 
#Aplicando grid search#
param_grid = {'bootstrap': [False], 'max_depth': [110], 'max_features': ['sqrt'], 'min_samples_leaf': [2, 3, 5], 'min_samples_split': [2, 5, 10], 'n_estimators': [200, 500, 700], 'criterion' : ['gini']}

#Creando modelo base#
rf_clf = RandomForestClassifier(random_state = 42)

grid_search = GridSearchCV(estimator = rf_clf, param_grid = param_grid, cv = 5,verbose = 2, n_jobs = -1, return_train_score=True)

grid_search.fit(X_train, y_train);

In [ ]:
#Mejores parametros de Grid Search#
grid_search.best_params_

In [ ]:
#Evaluacion del mejor modelo del Grid Search#
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
best_rf_clf = RandomForestClassifier(random_state = 42, max_depth = None, max_features = 110, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 1000, criterion = 'gini', bootstrap = False)

best_rf_clf.fit(X_train,y_train)

best_rf_prediction = best_rf_clf.predict(X_valid)
best_rf_accuracy = accuracy_score(y_valid,best_rf_prediction)
print("Training accuracy Score    : ",best_rf_clf.score(X_train,y_train))
print("Validation accuracy Score : ",best_rf_accuracy )
print(classification_report(best_rf_prediction,y_valid))

In [ ]:
#Prueba RF sin especificaciones#
rf_clf = RandomForestClassifier(random_state = 42)

rf_clf.fit(X_train,y_train)

rf_prediction = rf_clf.predict(X_valid)
rf_accuracy = accuracy_score(y_valid,rf_prediction)
print("Training accuracy Score    : ",rf_clf.score(X_train,y_train))
print("Validation accuracy Score : ",rf_accuracy )
print(classification_report(rf_prediction,y_valid))

In [ ]:
#Modelo Naive Bayesano multiclass#
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
naiveByes_clf = MultinomialNB()

naiveByes_clf.fit(X_train,y_train)

NB_prediction = naiveByes_clf.predict(X_valid)
NB_accuracy = accuracy_score(y_valid,NB_prediction)
print("training accuracy Score    : ",naiveByes_clf.score(X_train,y_train))
print("Validation accuracy Score : ",NB_accuracy )
print(classification_report(NB_prediction,y_valid))

In [ ]:
pprint(naiveByes_clf.get_params())

In [ ]:
#RandomSearch NB#
alpha = [0, 1, 2]
class_prior = [None]
fit_prior = [True, False]

nbr_grid = {'alpha': alpha, 'class_prior': class_prior, 'fit_prior': fit_prior}
nb_random = RandomizedSearchCV(estimator = naiveByes_clf, param_distributions = nbr_grid, n_iter = 100, cv = 5,verbose=2, random_state=42, n_jobs = -1)

nb_random.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
best_NB = MultinomialNB( fit_prior = False, class_prior = None, alpha = 0)

best_NB.fit(X_train,y_train)

best_NB_prediction = best_NB.predict(X_valid)
best_NB_accuracy = accuracy_score(y_valid,best_NB_prediction)
print("Training accuracy Score    : ",best_NB.score(X_train,y_train))
print("Validation accuracy Score : ",best_NB_accuracy )
print(classification_report(best_NB_prediction,y_valid))

In [ ]:
#SVM#
from sklearn.svm import SVC

svc = SVC(gamma = 'scale')

svc.fit(X_train, y_train)

svc_prediction = svc.predict(X_valid)
svc_accuracy = accuracy_score(y_valid,svc_prediction)
print("Training accuracy Score    : ",svc.score(X_train,y_train))
print("Validation accuracy Score : ",svc_accuracy )
print(classification_report(svc_prediction,y_valid))

In [ ]:
svc.get_params

In [ ]:
#RandomSearch SVM#
C = [1, 5, 10, 15]
class_weight = [None, 'balanced']
decision_function_shape = ['ovo']
kernel =['poly', 'rbf','sigmoid' ]
gamma = ['scale', 'auto']

svmr_grid = {'C': C, 'class_weight': class_weight, 'decision_function_shape': decision_function_shape, 'kernel': kernel, 'gamma': gamma}
svm_random = RandomizedSearchCV(estimator = svc, param_distributions = svmr_grid, n_iter = 100, cv = 5,verbose=2, random_state=42, n_jobs = -1)

svm_random.fit(X_train,y_train)

In [ ]:
svm_random.best_params_

In [ ]:
#Aplicando grid search#
param_grid_SVM = {'kernel': ['rbf'], 'gamma': ['scale'], 'decision_function_shape': ['ovo'], 'class_weight': [None], 'C': [2, 3, 4,5]}

#Creando modelo base#
SVM_clf = SVC(random_state = 42)

grid_search_SVM = GridSearchCV(estimator = SVM_clf, param_grid = param_grid_SVM, cv = 5,verbose = 2, n_jobs = -1, return_train_score=True)

grid_search_SVM.fit(X_train, y_train);

In [ ]:
grid_search_SVM.best_params_

In [ ]:
#Evaluacion del mejor modelo del Grid Search SVM#

best_SVM_clf = SVC(random_state = 42, kernel = 'rbf', gamma = 'scale', decision_function_shape = 'ovo', class_weight = None, C = 0.5)

best_SVM_clf.fit(X_train,y_train)

best_SVM_prediction = best_SVM_clf.predict(X_valid)
best_SVM_accuracy = accuracy_score(y_valid,best_SVM_prediction)
print("Training accuracy Score    : ",best_SVM_clf.score(X_train,y_train))
print("Validation accuracy Score : ",best_SVM_accuracy )
print(classification_report(best_SVM_prediction,y_valid))

In [ ]:
#Regresión Logística#
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

logreg_prediction = logreg.predict(X_valid)
logreg_accuracy = accuracy_score(y_valid,logreg_prediction)
print("Training accuracy Score    : ",logreg.score(X_train,y_train))
print("Validation accuracy Score : ",logreg_accuracy )
print(classification_report(logreg_prediction,y_valid))

In [ ]:
logreg.get_params

In [ ]:
#RandomSearch Regresión logistica#
C = [0, 1, 5, 10]
class_weight = [None, 'balanced']
solver = ['newton-cg', 'lbfgs', 'sag', 'saga']
penalty =['l2', None ]
warm_start = ['scale', 'auto']

svmr_grid = {'C': C, 'class_weight': class_weight, 'decision_function_shape': decision_function_shape, 'kernel': kernel, 'gamma': gamma}
svm_random = RandomizedSearchCV(estimator = svc, param_distributions = svmr_grid, n_iter = 100, cv = 5,verbose=2, random_state=42, n_jobs = -1)

svm_random.fit(X_train,y_train)